In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.readers.file import PyMuPDFReader

from IPython.display import Markdown, display

In [6]:
# Settings.embed_model = HuggingFaceEmbedding("BAAI/bge-m3")
Settings.embed_model = HuggingFaceEmbedding("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

<All keys matched successfully>


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [7]:
Settings.llm = Ollama(model="llama3", request_timeout=360.0)

In [14]:
import glob
pdf_loader = PyMuPDFReader()
base_path = 'data'

pdfs = []
for file in glob.glob(base_path + '/*.pdf', recursive=True):
    pdfs.extend(pdf_loader.load_data(file))


In [18]:
documents = []
documents.extend(pdfs)

In [20]:
# import nest_asyncio

# nest_asyncio.apply()

# from llama_parse import LlamaParse

# parser = LlamaParse(
#     api_key="llx-Lue9DuJA7MvcUbapEeiEm3nZgHVYNs3LOF9IfAd61A4Hz9CU",
#     result_type="markdown",
#     verbose=True
# )
# index = VectorStoreIndex.from_documents(
#     # pdfs
#     SimpleDirectoryReader(
#         base_path,
#         file_extractor={'.pdf': parser}
#     ).load_data()
# )

Started parsing the file under job_id cac11eca-3670-4d49-ba0f-ba0f5e237dc6
Started parsing the file under job_id cac11eca-ea6d-45f3-ace8-0178bdc027f0
Started parsing the file under job_id b35a95d7-beb5-497f-8cbd-0567a2ec5d6a
.

In [19]:
from llama_index.core.node_parser import SentenceSplitter, HierarchicalNodeParser

node_parser = HierarchicalNodeParser.from_defaults()
nodes = node_parser.get_nodes_from_documents(documents)

In [20]:
# define storage context
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext

docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

In [21]:
## Load index into vector index
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes
from llama_index.core import VectorStoreIndex

leaf_nodes = get_leaf_nodes(nodes)
root_nodes = get_root_nodes(nodes)

base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
)

In [22]:
from llama_index.core.retrievers import AutoMergingRetriever

base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

In [23]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever)
base_query_engine = RetrieverQueryEngine.from_args(base_retriever)

In [21]:
# query_engine = index.as_query_engine()

In [24]:
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [30]:
engine = query_engine.as_chat_engine()

response = query_engine.query("O que é um laço?")
print(response)

response = base_query_engine.query("O que é um laço?")
print(response)

AttributeError: 'RetrieverQueryEngine' object has no attribute 'as_chat_engine'